# Medi-Core Gen AI: The Implementation Story

## 1. Introduction: From Zero to AI
This notebook explains the complete journey of building the **Medi-Core** application.
We built a **Multimodal Medical Assistant** that can:
1.  **See**: Analyze medical images (X-Rays, handwritten notes).
2.  **Read**: Understand complex PDF documents.
3.  **Chat**: Interact with doctors/patients in real-time.

---

## 2. Why Google Gemini? (The "Vision" Part)

**The Problem:** We wanted to upload an image and ask questions about it.
**The Solution:** We chose **Google Gemini** (specifically `gemini-flash-latest`).

### Why not other models?
-   **Multimodal Native:** Gemini was built to understand text, images, and video *at the same time*.
-   **Speed:** The "Flash" version is extremely fast, essential for a real-time chat app.
-   **Cost:** It offers a generous free tier for developers.

### Why API Keys?
You noticed we used `GOOGLE_API_KEY`. Why? 
-   **Access Control:** Google needs to know who is calling their expensive servers.
-   **Quotas:** To prevent one user from crashing the system, they count every request.
-   **Security:** This key acts like a password for your application to talk to the AI brain.

In [ ]:
# How we used the key in code (server/modules/gemini_handler.py)
import google.generativeai as genai
import os

# 1. Load the key from the environment (securely)
api_key = os.getenv("GOOGLE_API_KEY")

# 2. Authenticate
genai.configure(api_key=api_key)

# 3. Call the model
model = genai.GenerativeModel('gemini-flash-latest')

## 3. Why Vector Databases & Embeddings? (The "Brain" Part)

**The Problem:** An AI model (LLM) doesn't know about *your* specific PDF documents. It only knows what it learned from the internet 2 years ago.

**The Solution:** **RAG (Retrieval Augmented Generation)**.

### Step 1: Tokenization
Computers don't understand words like "Medicine". They understand numbers.
**Tokenization** breaks text into small pieces (tokens).
-   Input: "Aspirin cures headaches."
-   Tokens: `["Asp", "irin", "cures", "head", "aches", "."]`

### Step 2: Embeddings
We convert these tokens into a long list of numbers called a **Vector** (e.g., `[0.1, -0.5, 0.8, ...]`).
-   The magic is that similar concepts get similar numbers.
-   "Dog" and "Puppy" will be mathematically close.
-   "Dog" and "Car" will be far apart.

### Step 3: Vector Database (Pinecone)
We cannot search a million numbers with a standard SQL database.
**Pinecone** is a specialized database designed to find "nearest neighbors" (similar vectors) incredibly fast.

**Why we used it:**
-   When you ask "What are the side effects?", we convert that question to numbers.
-   We ask Pinecone: "Give me the 3 PDF pages that are mathematically closest to this question."
-   We send those 3 pages to the AI to write the answer.

In [ ]:
# Conceptual Code for Vector Search

# 1. Convert text to vector (Embedding)
query_vector = voyage_ai.embed("What are the side effects?")

# 2. Search Pinecone
results = pinecone_index.query(vector=query_vector, top_k=3)

# 3. Use the results
print("Found relevant info:", results['matches'])

## 4. Why Streamlit? (The "Face" of the App)

**The Problem:** Building a frontend in React, HTML, and CSS takes weeks.
**The Solution:** **Streamlit** allowed us to build the entire UI in Python in just a few hours.

### Key Features:
-   **State Management:** It handles the user's session automatically.
-   **Widgets:** File uploaders (`st.file_uploader`) and Chat inputs (`st.chat_input`) are pre-built.
-   **Custom CSS:** We injected HTML/CSS to give it that "Cyberpunk/Web3" look, overriding the default style.